In [7]:
resource = "../data/generated/"

In [8]:
import pandas as pd
from scipy.optimize import fmin_bfgs, basinhopping, differential_evolution
import scipy.optimize as opt 
from numpy import hstack, zeros, ones, array, mat, tile, reshape, squeeze, eye, asmatrix
from gmm import *

In [9]:
# Set the start and end dates of the analysis period
start_date = pd.to_datetime('1975-01-01')
end_date = pd.to_datetime('2008-04-30')

ts = pd.read_csv(resource + 'time_series_1.csv', parse_dates=['date'], index_col=['date'])
ts.index.freq = 'M'

# Set analysis period
ts1 = ts.loc[(ts.index >= start_date) & (ts.index <= end_date)]

In [10]:
bm_port = pd.read_csv(resource + f'bm_port.csv', parse_dates=['date'], index_col=['date'])
ts3 = pd.merge(ts1, bm_port, on='date', how='left')
macro_factors = ts3[['myp','ui','dsv','ats','sts','fx']].values
financial_factors = ts3[['ex_mkt','smb','hml','mom']].values
riskfree = ts3['rf'].values
portfolios = ts3[['dec_1','dec_2','dec_3','dec_4','dec_5','dec_6','dec_7','dec_8','dec_9','dec_10']].values
T,N = portfolios.shape
excessRet = portfolios - reshape(riskfree,(T,1))
K = np.size(macro_factors,1)

startingVals = zeros(6)
# startingVals = [
# -49.5247,
# -4.7737,
# 113.7766,
# 25.9245,
# 90.8717,
# 16.7917
# ]
# startingVals = [
# 1000,
# 1000,
# 1000,
# 1000,
# 1000,
# 1000
#  ]

Winv = eye(N)
args = (excessRet, macro_factors, Winv)
iteration = 0
lastValue = 0
functionCount = 0
# fit_gmm = fmin_bfgs(gmm_objective_b, startingVals, args=(excessRet, macro_factors, eye(N)), callback=iter_print)
# fit_gmm = basinhopping(gmm_objective_b, startingVals, minimizer_kwargs={'args': args})
# fit_gmm = differential_evolution(gmm_objective_b, bounds=[(1e-10, 1000), (1e-10, 1000), (1e-10, 1000), (1e-10, 1000), (1e-10, 1000), (1e-10, 1000)], args=args)
fit_gmm = opt.minimize(gmm_objective_b, startingVals, args=(excessRet, macro_factors, eye(N)),
                       method='BFGS', options={'gtol': 1e-10, 'maxiter': 1000})
# fit_gmm = opt.minimize(gmm_objective_b, startingVals, args=(excessRet, macro_factors, eye(N)),
#                        method='Nelder-Mead')
# fit_gmm = opt.minimize(gmm_objective_b, startingVals, args=(excessRet, macro_factors, eye(N)), callback=iter_print, method='L-BFGS-B')
fit_gmm

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 0.001689611555707051
        x: [-3.604e+01  1.696e+02  8.137e+01 -1.558e+02  3.320e+01
            -6.113e+00]
      nit: 139
      jac: [-1.150e-09  4.802e-10 -1.746e-10  1.019e-10  6.112e-10
             6.257e-10]
 hess_inv: [[ 1.755e+05  1.120e+05 ...  1.777e+05 -3.468e+04]
            [ 1.120e+05  8.404e+04 ...  9.996e+04 -2.345e+04]
            ...
            [ 1.777e+05  9.996e+04 ...  2.224e+05 -7.138e+04]
            [-3.468e+04 -2.345e+04 ... -7.138e+04  6.554e+04]]
     nfev: 1267
     njev: 181

In [11]:
import numpy as np
from statsmodels.sandbox.regression.gmm import GMM

class estimate_b(GMM):
    def momcond(self, params):
        fRets = self.exog
        pRets = self.instrument
        T,N = pRets.shape
        T,K = fRets.shape
        b = params
        b = squeeze(array(params)) 
        b = reshape(b,(K,1))
        sdf = ones((T,1)) - fRets @ b
        moments = pRets * kron(sdf,ones((1,N)))
        return moments

gmm_mod = estimate_b(endog=zeros(macro_factors.shape[0]), exog=macro_factors, instrument=excessRet)
gmm_fit8 = gmm_mod.fit(start_params=zeros(6), maxiter=1, inv_weights=eye(N), weights_method='hac', wargs={'maxlag':12}, optim_method='bfgs', optim_args={'gtol': 1e-10, 'maxiter': 1000})
gmm_fit8.summary()

Optimization terminated successfully.
         Current function value: 0.000004
         Iterations: 138
         Function evaluations: 174
         Gradient evaluations: 174


<class 'statsmodels.iolib.summary.Summary'>
"""
                              estimate_b Results                              
==============================================================================
Dep. Variable:                      y   Hansen J:                     0.001690
Model:                     estimate_b   Prob (Hansen J):                  1.00
Method:                           GMM                                         
Date:                Wed, 10 May 2023                                         
Time:                        14:52:38                                         
No. Observations:                 400                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1           -36.0391     54.753     -0.658      0.510    -143.352      71.274
x2           169.5382    202.575      0.837      0.403    -227.501     566.577
x3            81.3747     56.940      1.429      0.153     -30.225     192.974
x4          -155.8005    140.894     -1.106      0.269    -431.947     120.346
x5            33.2021     86.896      0.382      0.702    -137.111     203.515
x6            -6.1122     34.160     -0.179      0.858     -73.065      60.841
==============================================================================
"""

In [12]:
# b = ([ -49.5247,
# -4.7737,
# 113.7766,
# 25.9245,
# 90.8717,
# 16.7917])
# b = ([-46.47,  4.675,  108.4,  4.868,  94.44,
                #   12.19])
# b = ([ -50.01969851,  22.80054891, 107.80604282, -27.68159424,
#         85.04568974,   4.42476452])
# b = ([ -30.11158833,  237.66483381,   88.98292253, -148.44282895,
#          52.95375349,  -13.72251736])
b = ([ -2.448e+01,  2.359e+02,  8.662e+01, -1.550e+02,  6.203e+01,
            -1.383e+01])
# b = ([ 20.24410124, -1.89313026, 16.9439695 , -2.33814035, -1.56179756,
#        -2.41001749])
J = gmm_objective_b(b, excessRet, macro_factors, eye(N))
J

0.01571008063240392

In [ ]:
# class mimick(GMM):
#     def momcond(self, params):
#         x = self.exog
#         y = self.endog
#         T,K = x.shape
#         y = reshape(y,(T,1))
#         b = params
#         b = squeeze(array(params)) 
#         b = reshape(b,(K,1))
#         err = (y - x @ b)
#         moments = x * kron(err,ones((1,K)))
#         return moments

# # gmm_mod = mimick(endog=log_indprod_growth_nextyear, exog=macro_factors, instrument=None)
# # gmm_fit8 = gmm_mod.fit(start_params=zeros(23), maxiter=1, inv_weights=eye(N), weights_method='hac', wargs={'maxlag':11}, optim_method='bfgs', optim_args={'gtol': 1e-10, 'maxiter': 1000})
# # gmm_fit8.summary()

# mimick_exog = ts1[['ex_mkt',
#                    'ex_b10ret',
#                    'ex_b5ret',
#                 #    'ex_high_yd_bd_ret',
#                    'ex_gold_ret', 
#                    'slope_ex_mkt_87', 
#                    'slope_ex_mkt_9602', 
#                    'rf', 
#                    'lag_10y_3m_gov_bd_yd',
#                    'lag_1y_3m_gov_bd_yd',
#                    'lag_Baa_Aaa_bd_yd',
#                    'lag_sp_div_yd',
#                    'log_indprod_growth_lastyear',
#                    'infl_lastyear',
#                    'ex_mkt_lastyear',
#                    'lag_ex_mkt',
#                    'lag_ex_b10ret',
#                    'lag_ex_b5ret',
#                    'lag_ex_gold_ret',
#                    'lag_slope_ex_mkt_87',
#                    'lag_slope_ex_mkt_9602'
#                    ]]
# mimick_exog = sm.add_constant(mimick_exog)
# mimick_endog = ts1['log_indprod_growth_nextyear']
# T,K = mimick_exog.shape


# mimick_mod = mimick(endog=mimick_endog, exog=mimick_exog, instrument=None)
# mimick_fit = mimick_mod.fit(start_params=np.zeros(K), maxiter=1, inv_weights=np.eye(K), weights_method='hac', wargs={'maxlag':11}, optim_method='bfgs', optim_args={'gtol': 1e-12, 'maxiter': 1000})
# # gmm_fit8 = gmm_mod.fit(start_params=np.zeros(K), maxiter=1, inv_weights=np.eye(K), optim_method='bfgs', optim_args={'gtol': 1e-15, 'maxiter': 10000})
# mimick_fit.summary()

In [ ]:
# Get the one-year ahead industrial production growth expectatitions factor
# coef = mimickfull.params
# ts['myp'] = coef['ex_mkt'] * ts['ex_mkt'] + coef['ex_long_gov_ret'] * ts['ex_long_gov_ret'] + coef['ex_medium_gov_ret'] * ts['ex_medium_gov_ret'] + coef['ex_high_yd_bd_ret'] * ts['ex_high_yd_bd_ret'] + coef['ex_gold_ret'] * ts['ex_gold_ret'] + coef['slope_ex_mkt_87'] * ts['slope_ex_mkt_87'] + coef['slope_ex_mkt_9602'] * ts['slope_ex_mkt_9602']
# ts[f'myp{i}'] = coef['ex_mkt'] * ts['ex_mkt'] + coef['ex_b10ret'] * ts['ex_b10ret'] + coef['ex_b5ret'] * ts['ex_b5ret'] + coef['ex_gold_ret'] * ts['ex_gold_ret'] + coef['slope_ex_mkt_87'] * ts['slope_ex_mkt_87'] + coef['slope_ex_mkt_9602'] * ts['slope_ex_mkt_9602'] + coef['slope_ex_mkt_0709'] * ts['slope_ex_mkt_0709']

# def compute_myp(row):
#     if pd.isna(row['ex_high_yd_bd_ret']):
#         return coef['ex_mkt'] * row['ex_mkt'] + coef['ex_long_gov_ret'] * row['ex_long_gov_ret'] + coef['ex_medium_gov_ret'] * row['ex_medium_gov_ret'] + coef['ex_gold_ret'] * row['ex_gold_ret'] + coef['slope_ex_mkt_87'] * row['slope_ex_mkt_87'] + coef['slope_ex_mkt_9602'] * row['slope_ex_mkt_9602']
#     else:
#         return coef['ex_mkt'] * row['ex_mkt'] + coef['ex_long_gov_ret'] * row['ex_long_gov_ret'] + coef['ex_medium_gov_ret'] * row['ex_medium_gov_ret'] + coef['ex_high_yd_bd_ret'] * row['ex_high_yd_bd_ret'] + coef['ex_gold_ret'] * row['ex_gold_ret'] + coef['slope_ex_mkt_87'] * row['slope_ex_mkt_87'] + coef['slope_ex_mkt_9602'] * row['slope_ex_mkt_9602']
    
# def compute_myp(row):
#     if pd.isna(row['ex_high_yd_bd_ret']):
#         return coef['ex_mkt'] * row['ex_mkt'] + coef['ex_b10ret'] * row['ex_b10ret'] + coef['ex_b5ret'] * row['ex_b5ret'] + coef['ex_gold_ret'] * row['ex_gold_ret'] + coef['slope_ex_mkt_87'] * row['slope_ex_mkt_87'] + coef['slope_ex_mkt_9602'] * row['slope_ex_mkt_9602']
#     else:
#         return coef['ex_mkt'] * row['ex_mkt'] + coef['ex_b10ret'] * row['ex_b10ret'] + coef['ex_b5ret'] * row['ex_b5ret'] + coef['ex_high_yd_bd_ret'] * row['ex_high_yd_bd_ret'] + coef['ex_gold_ret'] * row['ex_gold_ret'] + coef['slope_ex_mkt_87'] * row['slope_ex_mkt_87'] + coef['slope_ex_mkt_9602'] * row['slope_ex_mkt_9602']

# ts['myp'] = ts.apply(compute_myp, axis=1)

In [ ]:
# save the complete dataset
ts.to_csv(results + 'time_series_1.csv')

In [ ]:
# i = 2
# ts_var = ts.loc[(ts.index >= sdate[i] + MonthBegin(-1)) & (ts.index <= edate[i])]
# print(sdate[i] + MonthBegin(-1))
# print(sdate[i])

# # Var (without gmm)
# var = VAR(ts_var[['hml','smb','mom',f'myp{i}','ui','dsv','ats','sts','fx']]).fit(maxlags=1, trend='c')
# print(var.summary())

In [ ]:
# %store -r premia_port
# %store -r premia_t_stat_port

# # Plot market price of risk
# row_name = ([r'Market price of risk ($\lambda$) (\times 100)'])

# col_name = (['MYP',
#              'UI', 
#              'DSV', 
#              'ATS',
#              'STS', 
#              'FX'])

# tex = []
# for index, i in enumerate([0,1,2]): 
#     premia_table = table_to_latex(pd.DataFrame(premia_port[i] * 100), pd.DataFrame(premia_t_stat_port[i]), col_name, row_name)
#     tex.append(premia_table)

# # Create files for tables and graphs
# for index, t in enumerate(tex):
#     # Open a .tex file for writing
#     with open(results + tables + f'riskpremia{index+1}.tex', 'w') as f:
#         # Write the LaTeX string to the file
#         f.write(t)

In [ ]:
%store -r lambeta_params2
%store -r lambeta_bse2
%store -r lambeta_tvalues2
%store -r lambeta_pvalues2

macro_factors = ['MYP', 'UI', 'DSV', 'ATS', 'STS', 'FX']
portfolios = ['BM deciles', 'Size deciles', 'Momentum deciles','Profit deciles','Invest deciles','Accruals deciles']

for index, j in enumerate([0,1,2,3,4,5]):
        # Create a figure with subplots for each column
        fig, axs = plt.subplots(1, 6, figsize=(40, 5))
        # Generate random data for example
        coefficients = lambeta_params2[j][17:-6].reshape((10,6))
        # coefficients = np.arange(1, beta.shape[0]+1)
        t_stats = lambeta_tvalues2[j][17:-6].reshape((10,6))
        # t_stats = np.arange(1, beta_t_stat.shape[0]+1)

        # Loop through each column and plot the histogram and line plot
        for index2, i in enumerate(range(coefficients.shape[1])):
            # Plot histogram of coefficients on left y-axis
            axs[i].bar(np.arange(1, coefficients.shape[0]+1), coefficients[:, i], color='blue')
            axs[i].set_ylabel('Coefficients', color='blue', fontsize=15)
            axs[i].tick_params(axis='y', labelcolor='blue', labelsize=15)
            
            # Add line plot of t-stats on right y-axis
            axs2 = axs[i].twinx()
            axs2.plot(np.arange(1, t_stats.shape[0]+1), t_stats[:, i], color='red', linewidth=3)
            axs2.set_ylabel('T-Stats', color='red', fontsize=15)
            axs2.tick_params(axis='y', labelcolor='red', labelsize=15)
            
            # Set x-axis label and tick marks
            axs[i].set_xlabel(portfolios[index], fontsize=25)
            axs[i].set_xticks(np.arange(1, coefficients.shape[0]+1))
            axs[i].tick_params(axis='x', labelsize=15)
            axs[i].set_title(macro_factors[index2], fontsize=35)

        # Increase space between subplots
        fig.subplots_adjust(wspace=0.5)

        # Save the figure to the specified location
        fig.savefig(results + plots + f'betahist{index+1}_sample2.png', bbox_inches='tight')

In [ ]:
%store -r lambeta_params_list
%store -r lambeta_bse_list
%store -r lambeta_tvalues_list
%store -r lambeta_pvalues_list

macro_factors = ['MYP', 'UI', 'DSV', 'ATS', 'STS', 'FX']
portfolios = ['BM deciles', 'Size deciles', 'Momentum deciles','Profit deciles','Invest deciles','Accruals deciles']


for k in range(len(lambeta_params_list)):
    lambeta_params = lambeta_params_list[k]
    lambeta_tvalues = lambeta_tvalues_list[k]
    for index, j in enumerate([0,1,2,3,4,5]):
        # Create a figure with subplots for each column
        fig, axs = plt.subplots(1, 6, figsize=(40, 5))
        # Generate random data for example
        coefficients = lambeta_params[j][-66:-6].reshape((10,6))
        # coefficients = np.arange(1, beta.shape[0]+1)
        t_stats = lambeta_tvalues[j][-66:-6].reshape((10,6))
        # t_stats = np.arange(1, beta_t_stat.shape[0]+1)

        # Loop through each column and plot the histogram and line plot
        for index2, i in enumerate(range(coefficients.shape[1])):
            # Plot histogram of coefficients on left y-axis
            axs[i].bar(np.arange(1, coefficients.shape[0]+1), coefficients[:, i], color='blue')
            axs[i].set_ylabel('Coefficients', color='blue', fontsize=15)
            axs[i].tick_params(axis='y', labelcolor='blue', labelsize=15)
            
            # Add line plot of t-stats on right y-axis
            axs2 = axs[i].twinx()
            axs2.plot(np.arange(1, t_stats.shape[0]+1), t_stats[:, i], color='red', linewidth=3)
            axs2.set_ylabel('T-Stats', color='red', fontsize=15)
            axs2.tick_params(axis='y', labelcolor='red', labelsize=15)
            
            # Set x-axis label and tick marks
            axs[i].set_xlabel(portfolios[index], fontsize=25)
            axs[i].set_xticks(np.arange(1, coefficients.shape[0]+1))
            axs[i].tick_params(axis='x', labelsize=15)
            axs[i].set_title(macro_factors[index2], fontsize=35)

        # Increase space between subplots
        fig.subplots_adjust(wspace=0.5)

        # Save the figure to the specified location
        fig.savefig(results + plots + f'betahist{index+1}_sample{k}.png', bbox_inches='tight')

In [ ]:
# %store -r beta_port
# %store -r beta_t_stat_port

# macro_factors = ['MYP', 'UI', 'DSV', 'ATS', 'STS', 'FX']
# portfolios = ['BM deciles', 'Size deciles', 'Momentum deciles']

# for index, j in enumerate([0,1,2]):
#     # Create a figure with subplots for each column
#     fig, axs = plt.subplots(1, 6, figsize=(40, 5))
#     # Generate random data for example
#     coefficients = beta_port[j]
#     # coefficients = np.arange(1, beta.shape[0]+1)
#     t_stats = beta_t_stat_port[j]
#     # t_stats = np.arange(1, beta_t_stat.shape[0]+1)

#     # Loop through each column and plot the histogram and line plot
#     for index2, i in enumerate(range(coefficients.shape[1])):
#         # Plot histogram of coefficients on left y-axis
#         axs[i].bar(np.arange(1, coefficients.shape[0]+1), coefficients[:, i], color='blue')
#         axs[i].set_ylabel('Coefficients', color='blue', fontsize=15)
#         axs[i].tick_params(axis='y', labelcolor='blue', labelsize=15)
        
#         # Add line plot of t-stats on right y-axis
#         axs2 = axs[i].twinx()
#         axs2.plot(np.arange(1, t_stats.shape[0]+1), t_stats[:, i], color='red', linewidth=3)
#         axs2.set_ylabel('T-Stats', color='red', fontsize=15)
#         axs2.tick_params(axis='y', labelcolor='red', labelsize=15)
        
#         # Set x-axis label and tick marks
#         axs[i].set_xlabel(portfolios[index], fontsize=25)
#         axs[i].set_xticks(np.arange(1, coefficients.shape[0]+1))
#         axs[i].tick_params(axis='x', labelsize=15)
#         axs[i].set_title(macro_factors[index2], fontsize=35)

#     # Increase space between subplots
#     fig.subplots_adjust(wspace=0.5)

#     # Save the figure to the specified location
#     fig.savefig(results + plots + f'betahist{index+1}.png', bbox_inches='tight')

In [ ]:
class step1_lambeta(GMM):
    def momcond(self, params):
        fRets = self.exog
        pRets = self.endog
        coeff = squeeze(array(params)) 
        T,P = pRets.shape
        T,M = fRets.shape
        beta = squeeze(array(coeff[:(P*M)]))
        lam = squeeze(array(coeff[(P*M):]))
        beta = reshape(beta,(P,M))
        lam = reshape(lam,(M,1))
        betalam = beta @ lam
        expectedRet = fRets @ beta.T
        e = pRets - expectedRet
        moments_beta = kron(e,ones((1,M))) * tile(fRets,P)     # E[(R^{ex,i} - beta^i*FF)*FF]=0 (orthogon. conditions for the time series regression) 
        moments_lam = pRets - betalam.T    # E[R^{ex,i} – beta^i*lambda] = 0 (pricing equations using the MPR)
        moments = hstack((moments_beta,moments_lam))

        return moments
    
class gmm_lambeta(GMM):
    def momcond(self, params):
        global nbase
        fRets = self.exog
        pRets = self.endog
        mimick = self.instrument
        coeff = squeeze(array(params)) 

        # first stage
        mimick_endog = mimick[:,0]
        mimick_exog = mimick[:,1:]
        mimick_exog = sm.add_constant(mimick_exog)
        T,K = mimick_exog.shape
        mimick_exog = reshape(mimick_exog,(T,K))
        mimick_endog = reshape(mimick_endog,(T,1))
        mimick_coeff = coeff[:K]
        mimick_coeff = reshape(mimick_coeff,(K,1))
        mimick_err = (mimick_endog - mimick_exog @ mimick_coeff)
        moments1 = mimick_exog * kron(mimick_err,ones((1,K)))

        # compute myp
        myp = mimick_exog[:,1:(nbase+1)] @ mimick_coeff[1:(nbase+1)] 

        # gmm
        full_fRets = column_stack((myp, fRets))
        T,P = pRets.shape
        T,M = full_fRets.shape
        betalam_params = params[K:]
        beta = squeeze(array(betalam_params[:(P*M)]))
        lam = squeeze(array(betalam_params[(P*M):]))
        beta = reshape(beta,(P,M))
        lam = reshape(lam,(M,1))
        betalam = beta @ lam
        expectedRet = full_fRets @ beta.T
        e = pRets - expectedRet
        moments_beta = kron(e,ones((1,M))) * tile(full_fRets,P)     # E[(R^{ex,i} - beta^i*FF)*FF]=0 (orthogon. conditions for the time series regression) 
        moments_lam = pRets - betalam.T    # E[R^{ex,i} – beta^i*lambda] = 0 (pricing equations using the MPR)
        moments2 = hstack((moments_beta,moments_lam))

        return column_stack((moments1,moments2))

# function to compute the derivative of the moments with respect to the parameters
def gmm_G(params, p_rets, f_rets):
    t,n = p_rets.shape
    t,k = f_rets.shape
    beta = squeeze(array(params[:(n*k)]))
    lam = squeeze(array(params[(n*k):]))
    beta = reshape(beta,(n,k))
    lam = reshape(lam,(k,1))
    G = np.zeros((n*k+k,n*k+n))
    ffp = (f_rets.T @ f_rets) / t
    G[:(n*k),:(n*k)] = kron(eye(n),ffp)
    G[:(n*k),(n*k):] = kron(eye(n),-lam)
    G[(n*k):,(n*k):] = -beta.T
    
    return G

# Add the derivative of the moment conditions from the mimicking portfolio estimate
def add_mimick_deriv(G, mimick):
    mimick = sm.add_constant(mimick)
    t,k = mimick.shape
    i,j = G.shape
    D = np.zeros((k+i,k+j))
    D[:k,:k] = (mimick.T @ mimick) / t
    D[k:,k:] = G
    return D.T

In [ ]:
macro_exogcolnames = ['ui','dsv','ats','sts','fx']

In [ ]:
lambeta_est_list = []
lambeta_params_list = [] 
lambeta_bse_list = [] 
lambeta_tvalues_list = []
lambeta_pvalues_list = [] 

for i in range(len(sdate)):
    # if i == 0:
    #     continue
    lambeta_est = []
    lambeta_params = [] 
    lambeta_bse = [] 
    lambeta_tvalues = []
    lambeta_pvalues = [] 

    for index, port in enumerate(['bm','size','mom','op','inv','ac']):

        ##############################################
        ######### Prepare the data ###################
        ##############################################

        # Merge with the portfolio return data
        ports_data = pd.read_csv(resource + f'{port}_port.csv', parse_dates=['date'], index_col=['date'])
        ts4 = pd.merge(ts, ports_data, on='date', how='left')

        # Set the start and end dates of the analysis period
        ts4 = ts4.loc[(ts4.index >= sdate[i]) & (ts4.index <= edate[i])]

        # set mimick portfolio variables for the subsample
        nbase = 7
        mimick_colnames_sample = mimick_colnames
        if (ts4['slope_ex_mkt_87'] == 0).all():
            mimick_colnames_sample = [colname for colname in mimick_colnames_sample if 'slope_ex_mkt_87' not in colname]
            nbase -= 1
        if (ts4['slope_ex_mkt_9602'] == 0).all():
            mimick_colnames_sample = [colname for colname in mimick_colnames_sample if 'slope_ex_mkt_9602' not in colname]
            nbase -= 1
        if (ts4['slope_ex_mkt_0709'] == 0).all():
            mimick_colnames_sample = [colname for colname in mimick_colnames_sample if 'slope_ex_mkt_0709' not in colname]
            nbase -= 1
        mimick = ts4[mimick_colnames_sample].values

        # Set variables and dimensions
        T,K = mimick.shape
        exog_macro_factors = ts4[macro_exogcolnames].values
        T,M = exog_macro_factors.shape
        myp = ts4[[f'myp{i}']].values
        riskfree = ts4['rf'].values
        portfolios = ts4[['dec_1','dec_2','dec_3','dec_4','dec_5','dec_6','dec_7','dec_8','dec_9','dec_10']].values
        T,P = portfolios.shape
        excessRet = portfolios - reshape(riskfree,(T,1))

        ##############################################
        ######### First stage ########################
        ##############################################

        # Starting values for the factor loadings and rick premia are estimated using OLS and simple means.
        betas = []
        for j in range(P):
            res = sm.OLS(excessRet[:,j],sm.add_constant(column_stack((myp, exog_macro_factors)))).fit()
            betas.append(res.params[1:])
        avgReturn = excessRet.mean(axis=0)
        avgReturn.shape = P,1
        betas = array(betas)
        res = sm.OLS(avgReturn, betas).fit()
        riskPremia = res.params
        riskPremia.shape = M+1
        startingVals = np.concatenate((betas.flatten(),riskPremia))

        # Estimation    
        step1lambeta_mod = step1_lambeta(endog=excessRet, exog=column_stack((myp, exog_macro_factors)), instrument=None)
        step1lambeta_fit = step1lambeta_mod.fit(start_params=startingVals, 
                                                maxiter=1, 
                                                inv_weights=eye(P*(M+2)),
                                                weights_method='hac', 
                                                wargs={'maxlag':12}, 
                                                optim_method='bfgs', 
                                                optim_args={'gtol': 1e-15, 'maxiter': 100000})
        # print(step1lambeta_fit.summary())

        ##############################################
        ######### Second stage #######################
        ##############################################

        # Compute the optimal weights from the first stage
        step1optW = step1lambeta_fit.model.calc_weightmatrix(step1lambeta_fit.model.momcond(step1lambeta_fit.params),
                                                        weights_method='hac',
                                                        wargs={'maxlag':12}
                                                        )

        # Compute the weights for the second stage 
        # (force parameters to be equal to parameters from estimating mimicking portfolio and gmm separately)
        Grad = step1lambeta_fit.model.gradient_momcond(params=step1lambeta_fit.params, epsilon=1e-4, centered=True).T
        npar = Grad.shape[0]
        A = zeros((K+npar, K+P*(M+2)))
        A[:K, :K] = eye(K)
        A[-npar:, -P*(M+2):] = Grad @ step1optW
        A = A * 1000000
        Winv = A.T @ A

        # Starting value 
        output1_array = np.array(mimick_list[i].params)
        output1_array_reshaped = output1_array.reshape(1, -1)
        output2_array = np.array(step1lambeta_fit.params)
        output2_array_reshaped = output2_array.reshape(1, -1)
        startVals = np.concatenate([output1_array_reshaped, output2_array_reshaped], axis=1)

        # Estimation    
        lambeta_mod = gmm_lambeta(endog=excessRet, exog=exog_macro_factors, instrument=mimick)
        lambeta_fit = lambeta_mod.fit(start_params=startVals, 
                                    maxiter=1, 
                                    inv_weights=Winv, 
                                    #   weights_method='hac', 
                                    #   wargs={'maxlag':12}, 
                                    optim_method='bfgs', 
                                    optim_args={'gtol': 1e-15, 'maxiter': 100000})
        # print(lambeta_fit.summary())

        # Append results for a certain subsample
        lambeta_est.append(lambeta_fit)
        lambeta_params.append(lambeta_fit.params)
        lambeta_bse.append(lambeta_fit.bse)
        lambeta_tvalues.append(lambeta_fit.tvalues)
        lambeta_pvalues.append(lambeta_fit.pvalues)
    
    # Append results for all subsample
    lambeta_est_list.append(lambeta_est)
    lambeta_params_list.append(lambeta_params)
    lambeta_bse_list.append(lambeta_bse)
    lambeta_tvalues_list.append(lambeta_tvalues)
    lambeta_pvalues_list.append(lambeta_pvalues)

%store lambeta_params_list
%store lambeta_bse_list
%store lambeta_tvalues_list
%store lambeta_pvalues_list

In [ ]:
for i in range(len(sdate)):
    if i == 0:
        continue
    if i == 1:
        continue
    lambeta_est2 = []
    lambeta_params2 = [] 
    lambeta_bse2 = [] 
    lambeta_tvalues2 = []
    lambeta_pvalues2 = [] 

    for index, port in enumerate(['bm','size','mom','op','inv','ac']):
    # for index, port in enumerate(['bm','size']):

        ##############################################
        ######### Prepare the data ###################
        ##############################################

        # Merge with the portfolio return data
        ports_data = pd.read_csv(resource + f'{port}_port.csv', parse_dates=['date'], index_col=['date'])
        ts4 = pd.merge(ts, ports_data, on='date', how='left')

        # Set the start and end dates of the analysis period
        ts4 = ts4.loc[(ts4.index >= sdate[i]) & (ts4.index <= edate[i])]

        # set mimick portfolio variables for the subsample
        nbase = 6
        mimick_colnames_sample = mimick_colnames
        if (ts4['slope_ex_mkt_87'] == 0).all():
            mimick_colnames_sample = [colname for colname in mimick_colnames_sample if 'slope_ex_mkt_87' not in colname]
            nbase -= 1
        if (ts4['slope_ex_mkt_9602'] == 0).all():
            mimick_colnames_sample = [colname for colname in mimick_colnames_sample if 'slope_ex_mkt_9602' not in colname]
            nbase -= 1
        mimick = ts4[mimick_colnames_sample].values

        # Set variables and dimensions
        T,K = mimick.shape
        exog_macro_factors = ts4[macro_exogcolnames].values
        T,M = exog_macro_factors.shape
        myp = ts4[[f'myp{i}']].values
        riskfree = ts4['rf'].values
        portfolios = ts4[['dec_1','dec_2','dec_3','dec_4','dec_5','dec_6','dec_7','dec_8','dec_9','dec_10']].values
        T,P = portfolios.shape
        excessRet = portfolios - reshape(riskfree,(T,1))

        ##############################################
        ######### First stage ########################
        ##############################################

        # Starting values for the factor loadings and rick premia are estimated using OLS and simple means.
        betas = []
        for j in range(P):
            res = sm.OLS(excessRet[:,j],sm.add_constant(column_stack((myp, exog_macro_factors)))).fit()
            betas.append(res.params[1:])
        avgReturn = excessRet.mean(axis=0)
        avgReturn.shape = P,1
        betas = array(betas)
        res = sm.OLS(avgReturn, betas).fit()
        riskPremia = res.params
        riskPremia.shape = M+1
        startingVals = np.concatenate((betas.flatten(),riskPremia))

        # Estimation    
        step1lambeta_mod = step1_lambeta(endog=excessRet, exog=column_stack((myp, exog_macro_factors)), instrument=None)
        step1lambeta_fit = step1lambeta_mod.fit(start_params=startingVals, 
                                                maxiter=1, 
                                                inv_weights=eye(P*(M+2)),
                                                # weights_method='hac', 
                                                # wargs={'maxlag':12}, 
                                                optim_method='bfgs', 
                                                optim_args={'gtol': 1e-15, 'maxiter': 100000})
        # print(step1lambeta_fit.summary())

        ##############################################
        ######### Second stage #######################
        ##############################################

        # Compute the optimal weights from the first stage
        step1optW = step1lambeta_fit.model.calc_weightmatrix(step1lambeta_fit.model.momcond(step1lambeta_fit.params),
                                                        weights_method='hac',
                                                        wargs={'maxlag':12},
                                                        params=step1lambeta_fit.params)

        # Compute the weights for the second stage 
        # (force parameters to be equal to parameters from estimating mimicking portfolio and gmm separately)
        # G = gmm_G(step1lambeta_fit.params*1000, excessRet*1000, column_stack((myp, exog_macro_factors))*1000)
        # D = add_mimick_deriv(G, mimick[:,1:]*1000)
        # npar = G.shape[0]
        # A = zeros((K+npar, K+P*(M+2)))
        # A[:K, :K] = eye(K) * 1000
        # A[-npar:, -P*(M+2):] = G @ (step1optW * 1000)
        # # W1 = eye(K+P*(M+2)) * 1000 - D @ inv(A @ D) @ A
        # # S = zeros((K+P*(M+2), K+P*(M+2)))
        # # S[:K, :K] = eye(K) * 1000
        # # S[K:, K:] = step1optW * 1000
        # # Winv = W1 @ S @ W1.T 
        # Winv = A.T @ A
        Grad = step1lambeta_fit.model.gradient_momcond(params=step1lambeta_fit.params, 
                                                       epsilon=1e-4, 
                                                       centered=True
                                                       ).T
        npar = Grad.shape[0]
        A = zeros((K+npar, K+P*(M+2)))
        A[:K, :K] = eye(K)
        A[-npar:, -P*(M+2):] = Grad @ step1optW
        A = A * 1000000
        Winv = A.T @ A

        # Starting value 
        output1_array = np.array(mimick_list[i].params)
        output1_array_reshaped = output1_array.reshape(1, -1)
        output2_array = np.array(step1lambeta_fit.params)
        output2_array_reshaped = output2_array.reshape(1, -1)
        startVals = np.concatenate([output1_array_reshaped, output2_array_reshaped], axis=1)

        # Estimation    
        lambeta_mod = gmm_lambeta(endog=excessRet, exog=exog_macro_factors, instrument=mimick)
        lambeta_fit = lambeta_mod.fit(start_params=startVals, 
                                    maxiter=1, 
                                    inv_weights=Winv, 
                                    #   weights_method='hac', 
                                    #   wargs={'maxlag':12}, 
                                    optim_method='bfgs', 
                                    optim_args={'gtol': 1e-15, 'maxiter': 100000})
        # print(lambeta_fit.summary())

        # Append results for a certain subsample
        lambeta_est2.append(lambeta_fit)
        lambeta_params2.append(lambeta_fit.params)
        lambeta_bse2.append(lambeta_fit.bse)
        lambeta_tvalues2.append(lambeta_fit.tvalues)
        lambeta_pvalues2.append(lambeta_fit.pvalues)

%store lambeta_params2
%store lambeta_bse2
%store lambeta_tvalues2
%store lambeta_pvalues2

In [ ]:
i = 2
port = 'bm'
# Merge with the portfolio return data
ports_data = pd.read_csv(resource + f'{port}_port.csv', parse_dates=['date'], index_col=['date'])
ts4 = pd.merge(ts, ports_data, on='date', how='left')

# Set the start and end dates of the analysis period
ts4 = ts4.loc[(ts4.index >= sdate[i]) & (ts4.index <= edate[i])]

# set mimick portfolio variables for the subsample
nbase = 6
mimick_colnames_sample = mimick_colnames
if (ts4['slope_ex_mkt_87'] == 0).all():
            mimick_colnames_sample = [colname for colname in mimick_colnames_sample if 'slope_ex_mkt_87' not in colname]
            nbase -= 1
if (ts4['slope_ex_mkt_9602'] == 0).all():
            mimick_colnames_sample = [colname for colname in mimick_colnames_sample if 'slope_ex_mkt_9602' not in colname]
            nbase -= 1
mimick = ts4[mimick_colnames_sample].values

T,K = mimick.shape
exog_macro_factors = ts4[macro_exogcolnames].values
T,M = exog_macro_factors.shape
myp = ts4[[f'myp{i}']].values
riskfree = ts4['rf'].values
portfolios = ts4[['dec_1','dec_2','dec_3','dec_4','dec_5','dec_6','dec_7','dec_8','dec_9','dec_10']].values
T,P = portfolios.shape
excessRet = portfolios - reshape(riskfree,(T,1))

        ##############################################
        ######### First stage ########################
        ##############################################

        # Starting values for the factor loadings and rick premia are estimated using OLS and simple means.
betas = []
for j in range(P):
            res = sm.OLS(excessRet[:,j],sm.add_constant(column_stack((myp, exog_macro_factors)))).fit()
            betas.append(res.params[1:])
avgReturn = excessRet.mean(axis=0)
avgReturn.shape = P,1
betas = array(betas)
res = sm.OLS(avgReturn, betas).fit()
riskPremia = res.params
riskPremia.shape = M+1
startingVals = np.concatenate((betas.flatten(),riskPremia))

        # Estimation    
step1lambeta_mod = step1_lambeta(endog=excessRet, exog=column_stack((myp, exog_macro_factors)), instrument=None)
step1lambeta_fit = step1lambeta_mod.fit(start_params=startingVals, 
                                                maxiter=1, 
                                                inv_weights=eye(P*(M+2)),
                                                # weights_method='hac', 
                                                # wargs={'maxlag':12}, 
                                                optim_method='bfgs', 
                                                optim_args={'gtol': 1e-15, 'maxiter': 100000})

In [ ]:
step1optW = step1lambeta_fit.model.calc_weightmatrix(step1lambeta_fit.model.momcond(step1lambeta_fit.params),
                                                        weights_method='hac',
                                                        wargs={'maxlag':12}
                                                        # params=step1lambeta_fit.params
                                                        )
# step1optW = eye(step1optW.shape[0])

        # Compute the weights for the second stage 
        # (force parameters to be equal to parameters from estimating mimicking portfolio and gmm separately)
Grad = step1lambeta_fit.model.gradient_momcond(params=step1lambeta_fit.params, epsilon=1e-4, centered=True).T
npar = Grad.shape[0]
A = zeros((K+npar, K+P*(M+2)))
A[:K, :K] = eye(K)
A[-npar:, -P*(M+2):] = Grad @ step1optW
# A[-npar:, -P*(M+2):] = Grad @ eye(P*(M+2))
A = A * 1000000  # number are too low
Winv = A.T @ A

In [ ]:
# Starting value 
output1_array = np.array(mimick_list[i].params)
output1_array_reshaped = output1_array.reshape(1, -1)
output2_array = np.array(step1lambeta_fit.params)
output2_array_reshaped = output2_array.reshape(1, -1)
startVals = np.concatenate([output1_array_reshaped, output2_array_reshaped], axis=1)

In [ ]:
# Estimation    
lambeta_mod = gmm_lambeta(endog=excessRet, exog=exog_macro_factors, instrument=mimick)
lambeta_fit = lambeta_mod.fit(start_params=startVals, 
                                    maxiter=0, 
                                    inv_weights=Winv,
                                    # weights_method='hac', 
                                    # wargs={'maxlag':12},
                                #     has_optimal_weights=False, 
                                    optim_method='bfgs', 
                                    optim_args={'gtol': 1e-15, 'maxiter': 100000})

In [ ]:
from scipy.optimize import fmin_bfgs, basinhopping, differential_evolution


def momcond(params,exog,endog,instrument):
        fRets = exog
        pRets = endog
        mimick = instrument
        coeff = squeeze(array(params)) 

        # first stage
        mimick_endog = mimick[:,0]
        mimick_exog = mimick[:,1:]
        mimick_exog = sm.add_constant(mimick_exog)
        T,K = mimick_exog.shape
        mimick_exog = reshape(mimick_exog,(T,K))
        mimick_endog = reshape(mimick_endog,(T,1))
        mimick_coeff = coeff[:K]
        mimick_coeff = reshape(mimick_coeff,(K,1))
        mimick_err = (mimick_endog - mimick_exog @ mimick_coeff)
        moments1 = mimick_exog * kron(mimick_err,ones((1,K)))

        # compute myp
        # myp = mimick_exog[:, 1:8] @ mimick_coeff[1:8]
        myp = mimick_exog[:, 1:7] @ mimick_coeff[1:7] 

        # gmm
        full_fRets = column_stack((myp, fRets))
        T,P = pRets.shape
        T,M = full_fRets.shape
        betalam_params = params[K:]
        beta = squeeze(array(betalam_params[:(P*M)]))
        lam = squeeze(array(betalam_params[(P*M):]))
        beta = reshape(beta,(P,M))
        lam = reshape(lam,(M,1))
        betalam = beta @ lam
        expectedRet = full_fRets @ beta.T
        e = pRets - expectedRet
        moments_beta = kron(e,ones((1,M))) * tile(full_fRets,P)     # E[(R^{ex,i} - beta^i*FF)*FF]=0 (orthogon. conditions for the time series regression) 
        moments_lam = pRets - betalam.T    # E[R^{ex,i} – beta^i*lambda] = 0 (pricing equations using the MPR)
        moments2 = hstack((moments_beta,moments_lam))

        return column_stack((moments1,moments2))

def momcond_mean(params,exog,endog,instrument):
        momcond1 = momcond(params,exog,endog,instrument)
        nobs_moms, k_moms = momcond1.shape
        return momcond1.mean(0)

def gmmobjective(params,weights,exog,endog,instrument):
        moms = momcond_mean(params,exog,endog,instrument)
        return np.dot(np.dot(moms, weights), moms)

optimizer = fmin_bfgs
weights = np.linalg.pinv(Winv)

params=optimizer(gmmobjective, startVals, args=(weights,exog_macro_factors,excessRet,mimick), gtol= 1e-15)

In [ ]:
import statsmodels.stats.sandwich_covariance as smcov
from statsmodels.tools.numdiff import approx_fprime


def calc_weightmatrix(moms, weights_method='cov', wargs=(),
                          params=None):
        nobs, k_moms = moms.shape

        centered = not ('centered' in wargs and not wargs['centered'])
        if not centered:
            # caller does not want centered moment conditions
            moms_ = moms
        else:
            moms_ = moms - moms.mean()
        if weights_method == 'cov':
            w = np.dot(moms_.T, moms_)
            if 'ddof' in wargs:
                # caller requests degrees of freedom correction
                if wargs['ddof'] == 'k_params':
                    w /= (nobs - self.k_params)
                else:
                    w /= (nobs - wargs['ddof'])
            else:
                # default: divide by nobs
                w /= nobs

        elif weights_method == 'flatkernel':
            #uniform cut-off window
            # This was a trial version, can use HAC with flatkernel
            if 'maxlag' not in wargs:
                raise ValueError('flatkernel requires maxlag')

            maxlag = wargs['maxlag']
            h = np.ones(maxlag + 1)
            w = np.dot(moms_.T, moms_)/nobs
            for i in range(1,maxlag+1):
                w += (h[i] * np.dot(moms_[i:].T, moms_[:-i]) / (nobs-i))

        elif weights_method == 'hac':
            maxlag = wargs['maxlag']
            if 'kernel' in wargs:
                weights_func = wargs['kernel']
            else:
                weights_func = smcov.weights_bartlett
                wargs['kernel'] = weights_func

            w = smcov.S_hac_simple(moms_, nlags=maxlag,
                                   weights_func=weights_func)
            w /= nobs #(nobs - self.k_params)

        elif weights_method == 'iid':
            # only when we have instruments and residual mom = Z * u
            # TODO: problem we do not have params in argument
            #       I cannot keep everything in here w/o params as argument
            u = self.get_error(params)

            if centered:
                # Note: I'm not centering instruments,
                #    should not we always center u? Ok, with centered as default
                u -= u.mean(0)  #demean inplace, we do not need original u

            instrument = self.instrument
            w = np.dot(instrument.T, instrument).dot(np.dot(u.T, u)) / nobs
            if 'ddof' in wargs:
                # caller requests degrees of freedom correction
                if wargs['ddof'] == 'k_params':
                    w /= (nobs - self.k_params)
                else:
                    # assume ddof is a number
                    w /= (nobs - wargs['ddof'])
            else:
                # default: divide by nobs
                w /= nobs

        else:
            raise ValueError('weight method not available')

        return w

def gradient_momcond(momcond_mean, params, epsilon=1e-4, centered=True):

        momcond = momcond_mean

        # TODO: approx_fprime has centered keyword
        if centered:
            gradmoms = (approx_fprime(params, momcond, epsilon=epsilon) +
                    approx_fprime(params, momcond, epsilon=-epsilon))/2
        else:
            gradmoms = approx_fprime(params, momcond, epsilon=epsilon)

        return gradmoms


omegahat = calc_weightmatrix(momcond(params,exog_macro_factors,excessRet,mimick), weights_method='cov', wargs=(),
                          params=params)

# gradmoms = gradient_momcond(momcond_mean(params,exog_macro_factors,excessRet,mimick),params, epsilon=1e-4, centered=True)
gradmoms = approx_fprime(params, momcond_mean, kwargs={'exog':exog_macro_factors,'endog':excessRet,'instrument':mimick}, epsilon=1e-4)

# cov = np.linalg.inv(np.dot(gradmoms.T,
#                     np.dot(np.linalg.inv(omegahat), gradmoms)))

cov = (np.dot(gradmoms.T,
                    np.dot(np.linalg.inv(omegahat), gradmoms)))

In [ ]:
covparams = lambeta_fit.model.momcond(lambeta_fit.params)
covgradmoms = lambeta_fit.model.gradient_momcond(params=lambeta_fit.params, epsilon=1e-4, centered=True)

lambeta_fit.calc_cov_params(covparams,
                            covgradmoms,
                            has_optimal_weights=True
                            )

In [ ]:
import matplotlib.pyplot as plt
index = 1

plots = "plots/"

macro_factors = ['MYP', 'UI', 'DSV', 'ATS', 'STS', 'FX']
portfolios = ['BM deciles', 'Size deciles', 'Momentum deciles','Profit deciles','Invest deciles','Accruals deciles']
# Create a figure with subplots for each column
fig, axs = plt.subplots(1, 6, figsize=(40, 5))
        # Generate random data for example
coefficients = lambeta_fit.params[21:-6].reshape((10,6))
        # coefficients = np.arange(1, beta.shape[0]+1)
t_stats = lambeta_fit.tvalues[21:-6].reshape((10,6))
        # t_stats = np.arange(1, beta_t_stat.shape[0]+1)

        # Loop through each column and plot the histogram and line plot
for index2, l in enumerate(range(coefficients.shape[1])):
            # Plot histogram of coefficients on left y-axis
            axs[l].bar(np.arange(1, coefficients.shape[0]+1), coefficients[:, l], color='blue')
            axs[l].set_ylabel('Coefficients', color='blue', fontsize=15)
            axs[l].tick_params(axis='y', labelcolor='blue', labelsize=15)
            
            # Add line plot of t-stats on right y-axis
            axs2 = axs[l].twinx()
            axs2.plot(np.arange(1, t_stats.shape[0]+1), t_stats[:, l], color='red', linewidth=3)
            axs2.set_ylabel('T-Stats', color='red', fontsize=15)
            axs2.tick_params(axis='y', labelcolor='red', labelsize=15)
            
            # Set x-axis label and tick marks
            axs[l].set_xlabel(portfolios[index], fontsize=25)
            axs[l].set_xticks(np.arange(1, coefficients.shape[0]+1))
            axs[l].tick_params(axis='x', labelsize=15)
            axs[l].set_title(macro_factors[index2], fontsize=35)

        # Increase space between subplots
fig.subplots_adjust(wspace=0.5)

In [ ]:
# for index, port in enumerate(['bm','size','mom','op','inv','ac']):
    # for index, port in enumerate(['inv']):

        ##############################################
        ######### Prepare the data ###################
        # ##############################################
        # port = 'inv'
        # # Merge with the portfolio return data
        # ports_data = pd.read_csv(resource + f'{port}_port.csv', parse_dates=['date'], index_col=['date'])
        # ts4 = pd.merge(ts, ports_data, on='date', how='left')

        # # Set the start and end dates of the analysis period
        # ts4 = ts4.loc[(ts4.index >= sdate[i]) & (ts4.index <= edate[i])]

        # mimick = ts4[mimick_colnames].values
        # T,K = mimick.shape
        # exog_macro_factors = ts4[macro_exogcolnames].values
        # T,M = exog_macro_factors.shape
        # myp = ts4[[f'myp{i}']].values
        # riskfree = ts4['rf'].values
        # portfolios = ts4[['dec_1','dec_2','dec_3','dec_4','dec_5','dec_6','dec_7','dec_8','dec_9','dec_10']].values
        # T,P = portfolios.shape
        # excessRet = portfolios - reshape(riskfree,(T,1))

        # ##############################################
        # ######### First stage ########################
        # ##############################################

        # # Starting values for the factor loadings and rick premia are estimated using OLS and simple means.
        # betas = []
        # for j in range(P):
        #     res = sm.OLS(excessRet[:,j],sm.add_constant(column_stack((myp, exog_macro_factors)))).fit()
        #     betas.append(res.params[1:])
        # avgReturn = excessRet.mean(axis=0)
        # avgReturn.shape = P,1
        # betas = array(betas)
        # res = sm.OLS(avgReturn, betas).fit()
        # riskPremia = res.params
        # riskPremia.shape = M+1
        # startingVals = np.concatenate((betas.flatten(),riskPremia))

        # # Estimation    
        # step1lambeta_mod = step1_lambeta(endog=excessRet, exog=column_stack((myp, exog_macro_factors)), instrument=None)
        # step1lambeta_fit = step1lambeta_mod.fit(start_params=startingVals, 
        #                                         maxiter=1, 
        #                                         inv_weights=eye(P*(M+2)),
        #                                         weights_method='hac', 
        #                                         wargs={'maxlag':12}, 
        #                                         optim_method='bfgs', 
        #                                         optim_args={'gtol': 1e-15, 'maxiter': 100000})
        # print(step1lambeta_fit.summary())

        ##############################################
        ######### Second stage #######################
        ##############################################

        # Compute the optimal weights from the first stage
        step1optW = step1lambeta_fit.model.calc_weightmatrix(step1lambeta_fit.model.momcond(step1lambeta_fit.params),
                                                        weights_method='hac',
                                                        wargs={'maxlag':12},
                                                        params=step1lambeta_fit.params)

        # Compute the weights for the second stage 
        # (force parameters to be equal to parameters from estimating mimicking portfolio and gmm separately)
        G = gmm_G(step1lambeta_fit.params*10000, excessRet*10000, column_stack((myp, exog_macro_factors))*10000)
        D = add_mimick_deriv(G, mimick[:,1:]*10000)
        npar = G.shape[0]
        A = zeros((K+npar, K+P*(M+2)))
        A[:K, :K] = eye(K) * 10000
        A[-npar:, -P*(M+2):] = G @ (step1optW * 10000)
        # W1 = eye(K+P*(M+2)) * 1000 - D @ inv(A @ D) @ A
        # S = zeros((K+P*(M+2), K+P*(M+2)))
        # S[:K, :K] = eye(K) * 1000
        # S[K:, K:] = step1optW * 1000
        # Winv = W1 @ S @ W1.T 
        Winv = A.T @ A

        # Starting value 
        output1_array = np.array(mimick_list[i].params)
        output1_array_reshaped = output1_array.reshape(1, -1)
        output2_array = np.array(step1lambeta_fit.params)
        output2_array_reshaped = output2_array.reshape(1, -1)
        startVals = np.concatenate([output1_array_reshaped, output2_array_reshaped], axis=1)

        # Estimation    
        lambeta_mod = gmm_lambeta(endog=excessRet, exog=exog_macro_factors, instrument=mimick)
        lambeta_fit = lambeta_mod.fit(start_params=startVals, 
                                    maxiter=1, 
                                    inv_weights=Winv, 
                                    #   weights_method='hac', 
                                    #   wargs={'maxlag':12}, 
                                    optim_method='bfgs', 
                                    optim_args={'gtol': 1e-15, 'maxiter': 100000})
        # print(lambeta_fit.summary())

In [ ]:
# port = 'bm'
# ports_data = pd.read_csv(resource + f'{port}_port.csv', parse_dates=['date'], index_col=['date'])
# ts4 = pd.merge(ts, ports_data, on='date', how='left')

# # Set the start and end dates of the analysis period
# ts4 = ts4.loc[(ts4.index >= start_date) & (ts4.index <= end_date)]

# mimick = ts4[['log_indprod_growth_nextyear',
#                 'ex_mkt',
#                    'ex_b10ret',
#                    'ex_b5ret',
#                 #    'ex_high_yd_bd_ret',
#                    'ex_gold_ret', 
#                    'slope_ex_mkt_87', 
#                    'slope_ex_mkt_9602', 
#                    'rf', 
#                    'lag_10y_3m_gov_bd_yd',
#                    'lag_1y_3m_gov_bd_yd',
#                    'lag_Baa_Aaa_bd_yd',
#                    'lag_sp_div_yd',
#                    'log_indprod_growth_lastyear',
#                    'infl_lastyear',
#                    'ex_mkt_lastyear',
#                    'lag_ex_mkt',
#                    'lag_ex_b10ret',
#                    'lag_ex_b5ret',
#                    'lag_ex_gold_ret',
#                    'lag_slope_ex_mkt_87',
#                    'lag_slope_ex_mkt_9602'
#                    ]].values
# T,K = mimick.shape
# exog_macro_factors = ts4[['ui','dsv','ats','sts','fx']].values
# T,M = exog_macro_factors.shape
# myp = ts4[['myp']].values
# riskfree = ts4['rf'].values
# portfolios = ts4[['dec_1','dec_2','dec_3','dec_4','dec_5','dec_6','dec_7','dec_8','dec_9','dec_10']].values
# T,P = portfolios.shape
# excessRet = portfolios - reshape(riskfree,(T,1))

In [ ]:
# class step1_lambeta(GMM):
#     def momcond(self, params):
#         fRets = self.exog
#         pRets = self.endog
#         coeff = squeeze(array(params)) 

#         T,P = pRets.shape
#         T,M = fRets.shape
#         # var_coeff = reshape(var_coeff,(N,M))
#         # print(P, M)
#         beta = squeeze(array(coeff[:(P*M)]))
#         lam = squeeze(array(coeff[(P*M):]))
#         beta = reshape(beta,(P,M))
#         lam = reshape(lam,(M,1))
#         betalam = beta @ lam
#         expectedRet = fRets @ beta.T
#         e = pRets - expectedRet
#         # print(M, P)
#         moments_beta = kron(e,ones((1,M))) * tile(fRets,P)     # E[(R^{ex,i} - beta^i*FF)*FF]=0 (orthogon. conditions for the time series regression) 
#         moments_lam = pRets - betalam.T    # E[R^{ex,i} – beta^i*lambda] = 0 (pricing equations using the MPR)
#         moments = hstack((moments_beta,moments_lam))

#         return moments

# # Starting values for the factor loadings and rick premia are estimated using OLS and simple means.
# betas = []
# for i in range(P):
#     res = sm.OLS(excessRet[:,i],sm.add_constant(column_stack((myp, exog_macro_factors)))).fit()
#     betas.append(res.params[1:])
# avgReturn = excessRet.mean(axis=0)
# avgReturn.shape = P,1
# betas = array(betas)
# res = sm.OLS(avgReturn, betas).fit()
# riskPremia = res.params
# riskPremia.shape = M+1
# startingVals = np.concatenate((betas.flatten(),riskPremia))

# # Estimation    
# step1lambeta_mod = step1_lambeta(endog=excessRet, exog=column_stack((myp, exog_macro_factors)), instrument=None)
# step1lambeta_fit = step1lambeta_mod.fit(start_params=startingVals, 
#                                         maxiter=1, 
#                                         inv_weights=eye(P*(M+2)), 
#                                         weights_method='hac', 
#                                         wargs={'maxlag':12}, 
#                                         optim_method='bfgs', 
#                                         optim_args={'gtol': 1e-15, 'maxiter': 10000})
# step1lambeta_fit.summary()

In [ ]:
# optW = step1lambeta_fit.model.calc_weightmatrix(step1lambeta_fit.model.momcond(step1lambeta_fit.params),
#                                                     weights_method='hac',
#                                                     wargs={'maxlag':12},
#                                                     params=step1lambeta_fit.params)
# optW.shape

In [ ]:
# def gmm_G(params, p_rets, f_rets):
#     t,n = p_rets.shape
#     t,k = f_rets.shape
#     beta = squeeze(array(params[:(n*k)]))
#     lam = squeeze(array(params[(n*k):]))
#     beta = reshape(beta,(n,k))
#     lam = reshape(lam,(k,1))
#     G = np.zeros((n*k+k,n*k+n))
#     ffp = (f_rets.T @ f_rets) / t
#     G[:(n*k),:(n*k)] = kron(eye(n),ffp)
#     G[:(n*k),(n*k):] = kron(eye(n),-lam)
#     G[(n*k):,(n*k):] = -beta.T
#     return G

# def add_mimick_deriv(G, mimick):
#     mimick = sm.add_constant(mimick)
#     t,k = mimick.shape
#     i,j = G.shape
#     D = np.zeros((k+i,k+j))
#     D[:k,:k] = (mimick.T @ mimick) / t
#     D[k:,k:] = G
#     return D.T

# G = gmm_G(step1lambeta_fit.params*1000, excessRet*1000, column_stack((myp, exog_macro_factors))*1000)
# D = add_mimick_deriv(G, mimick[:,1:]*1000)
# # G = gmm_G(step1lambeta_fit.params, excessRet, column_stack((myp, exog_macro_factors)))
# # D = add_mimick_deriv(G, mimick[:,1:])
# npar = G.shape[0]
# A = zeros((K+npar, K+P*(M+2)))
# # A[:K, :K] = eye(K) * 1000000000
# # A[-npar:, -P*(M+2):] = G @ optW * 1000000000
# # A[:K, :K] = eye(K)
# A[:K, :K] = eye(K) * 1000
# # A[-npar:, -P*(M+2):] = G @ optW * 100
# # A[-npar:, -P*(M+2):] = G @ (optW * 1000)
# A[-npar:, -P*(M+2):] = G @ (eye(P*(M+2)) * 1000)

# W1 = eye(K+P*(M+2)) * 1000 - D @ inv(A @ D) @ A

# S = zeros((K+P*(M+2), K+P*(M+2)))
# S[:K, :K] = eye(K) * 1000
# # S[K:, K:] = optW * 100
# S[K:, K:] = optW * 1000

# # Winv = W1 @ S @ W1.T * 100000
# # Winv = W1 @ S @ W1.T 

# Winv = A.T @ A


In [ ]:
# class gmm_lambeta(GMM):
#     def momcond(self, params):
#         fRets = self.exog
#         pRets = self.endog
#         mimick = self.instrument
#         coeff = squeeze(array(params)) 
        
#         # first stage
#         mimick_endog = mimick[:,0]
#         mimick_exog = mimick[:,1:]
#         mimick_exog = sm.add_constant(mimick_exog)
#         T,K = mimick_exog.shape
#         mimick_exog = reshape(mimick_exog,(T,K))
#         mimick_endog = reshape(mimick_endog,(T,1))
#         mimick_coeff = coeff[:K]
#         mimick_coeff = reshape(mimick_coeff,(K,1))
#         mimick_err = (mimick_endog - mimick_exog @ mimick_coeff)
#         moments1 = mimick_exog * kron(mimick_err,ones((1,K)))

#         # compute myp
#         myp = mimick_exog[:, 1:7] @ mimick_coeff[1:7] 

#         # gmm
#         full_fRets = column_stack((myp, fRets))
#         T,P = pRets.shape
#         T,M = full_fRets.shape
#         betalam_params = params[K:]
#         # var_coeff = reshape(var_coeff,(N,M))
#         # print(P, M)
#         # print(betalam_params.shape)
#         beta = squeeze(array(betalam_params[:(P*M)]))
#         lam = squeeze(array(betalam_params[(P*M):]))
#         beta = reshape(beta,(P,M))
#         lam = reshape(lam,(M,1))
#         betalam = beta @ lam
#         expectedRet = full_fRets @ beta.T
#         e = pRets - expectedRet
#         # print(M, P)
#         moments_beta = kron(e,ones((1,M))) * tile(full_fRets,P)     # E[(R^{ex,i} - beta^i*FF)*FF]=0 (orthogon. conditions for the time series regression) 
#         moments_lam = pRets - betalam.T    # E[R^{ex,i} – beta^i*lambda] = 0 (pricing equations using the MPR)
#         moments2 = hstack((moments_beta,moments_lam))
#         # print(moments2.shape)

#         return column_stack((moments1,moments2))
    

# # Starting value 
# output2_array = np.array(step1lambeta_fit.params)
# output2_array_reshaped = output2_array.reshape(1, -1)
# startVals = np.concatenate([output1_array_reshaped, output2_array_reshaped], axis=1)
    
# lambeta_mod = gmm_lambeta(endog=excessRet, exog=exog_macro_factors, instrument=mimick)
# lambeta_fit = lambeta_mod.fit(start_params=startVals, 
#                               maxiter=1, 
#                               inv_weights=Winv, 
#                             #   weights_method='hac', 
#                             #   wargs={'maxlag':12}, 
#                               optim_method='bfgs', 
#                               optim_args={'gtol': 1e-15, 'maxiter': 100000})
# lambeta_fit.summary()

In [ ]:
# import pandas as pd
# import numpy as np
# from tabprintin.beautify import *
# import matplotlib.pyplot as plt
# from varname import nameof
# import os
# import subprocess

# macro_factors = ['MYP', 'UI', 'DSV', 'ATS', 'STS', 'FX']
# portfolios = ['BM deciles', 'Size deciles', 'Momentum deciles']

# # Create a figure with subplots for each column
# fig, axs = plt.subplots(1, 6, figsize=(40, 5))
# # Generate random data for example
# coefficients = reshape(lambeta_fit.params[21:-6],(10,6))
#     # coefficients = np.arange(1, beta.shape[0]+1)
# t_stats = reshape(lambeta_fit.tvalues[21:-6],(10,6))
#     # t_stats = np.arange(1, beta_t_stat.shape[0]+1)

#     # Loop through each column and plot the histogram and line plot
# for index2, i in enumerate(range(coefficients.shape[1])):
#         # Plot histogram of coefficients on left y-axis
#         axs[i].bar(np.arange(1, coefficients.shape[0]+1), coefficients[:, i], color='blue')
#         axs[i].set_ylabel('Coefficients', color='blue', fontsize=15)
#         axs[i].tick_params(axis='y', labelcolor='blue', labelsize=15)
        
#         # Add line plot of t-stats on right y-axis
#         axs2 = axs[i].twinx()
#         axs2.plot(np.arange(1, t_stats.shape[0]+1), t_stats[:, i], color='red', linewidth=3)
#         axs2.set_ylabel('T-Stats', color='red', fontsize=15)
#         axs2.tick_params(axis='y', labelcolor='red', labelsize=15)
        
#         # Set x-axis label and tick marks
#         axs[i].set_xlabel(portfolios[index], fontsize=25)
#         axs[i].set_xticks(np.arange(1, coefficients.shape[0]+1))
#         axs[i].tick_params(axis='x', labelsize=15)
#         axs[i].set_title(macro_factors[index2], fontsize=35)
    
#     # Increase space between subplots
# fig.subplots_adjust(wspace=0.5)